In [1]:
import pandas as pd
import numpy as np
import pyodbc 

def collect(sql):
    conn = pyodbc.connect('DSN=BP_DB;UID=bpuser;PWD=kotrabp')   # BPUSER2 에서 데이터 가져옴  // 만약 BPUSER1에서 데이터를 가져올경우 'DSN=BP_DB2'로 변경
    conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    conn.setencoding(encoding='utf-8')
    data= pd.read_sql_query(sql, conn)
    conn.close()
    return data

In [9]:
# 어떻게 할까?? 일단 사업데이터를 가져오는 함수부터 만들어야...


def make_biz_data():
    sql = ''' SELECT baseyear 기준년도, bizgbncd 사업구분코드, dtlbizname 사업명, biz_from 사업시작일, biz_to 사업종료일,
(SELECT DEPTNAME FROM scrm.TB_CO_DEPT WHERE DEPTCD = A.IMPLDEPTID)  수행부서, procstepname 사업수행단계, seltfname 선정여부명, APPLNO 신청번호, coid 참가KOTRA기업ID, bsno 참가기업사업자번호
FROM SCRM.TB_DA_APPHIST_MAIN A
WHERE appldate > sysdate - 2000;
'''
    data = collect(sql)
    data = data[~data['사업수행단계'].isin(['사업취소', '취소'])]
    data = data[~data['선정여부명'].isin(['미선정', '불참'])]
    
    biz_cat = pd.DataFrame({'사업구분코드':data['사업구분코드'].value_counts().index.to_list(),
        '사업구분':['기타사업', '수출상담회', '신규수출기업화', '글로벌역량진단', '전시회', '무역사절단',
        '사업파트너연결지원', '유료사업(e)', '지사화/공동물류', '설명회', '월드챔프', '비즈니스세일즈', '열린무역관', '해외공동물류', '프로젝트수주지원']})
    
    data = data.merge(biz_cat, how = 'left')
    data['사업시작일'] = pd.to_datetime(data['사업시작일'])
    data['사업종료일'] = pd.to_datetime(data['사업종료일'].replace({'9999-12-31':np.nan}))
    data['사업시작월'] = data['사업시작일'].apply(lambda x: str(x)[:7].replace('-', ''))
    
    data = data[['기준년도','사업구분코드','사업구분','사업명','사업시작일','사업종료일','수행부서','사업수행단계',
                 '선정여부명','신청번호','참가KOTRA기업ID','참가기업사업자번호']]
    
    return data
    

In [10]:
biz_df = make_biz_data()

In [7]:
biz_df['사업구분코드'].value_counts().index

Index(['OB', 'EF', 'EG', 'GC', 'EX', 'TM', 'RA', 'BR', 'VO', 'PT', 'WC', 'LO',
       'OT', 'BS', 'FP'],
      dtype='object')

In [11]:
biz_df[['사업구분코드', '사업구분']].drop_duplicates()

,사업구분코드,사업구분
0,PT,설명회
27,EX,전시회
125,EF,수출상담회
466,TM,무역사절단
534,BS,해외공동물류
535,BR,유료사업(e)
576,OB,기타사업
577,RA,사업파트너연결지원
1595,VO,지사화/공동물류
1704,EG,신규수출기업화


In [5]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664653 entries, 0 to 664652
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   기준년도         664653 non-null  object        
 1   사업구분코드       664653 non-null  object        
 2   사업구분         664653 non-null  object        
 3   사업명          664653 non-null  object        
 4   사업시작일        664653 non-null  datetime64[ns]
 5   사업종료일        655952 non-null  datetime64[ns]
 6   수행부서         482067 non-null  object        
 7   사업수행단계       664424 non-null  object        
 8   선정여부명        40051 non-null   object        
 9   신청번호         664653 non-null  object        
 10  참가KOTRA기업ID  664653 non-null  object        
 11  참가기업사업자번호    664653 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 65.9+ MB


In [6]:
biz_df.head()

,기준년도,사업구분코드,사업구분,사업명,사업시작일,사업종료일,수행부서,사업수행단계,선정여부명,신청번호,참가KOTRA기업ID,참가기업사업자번호
0,2018,PT,설명회,NADCAP HT 2018,2018-07-16,2018-07-19,제조산업팀,신청,참가,PT0200043824,EC115392,5158140973
1,2017,PT,설명회,MDB·글로벌 개발펀드 프로젝트 플라자(설명회_14일 오전),2017-11-14,2017-11-16,인프라 에너지산업팀,신청,참가,PT0200040761,1026547,1018116293
2,2017,PT,설명회,이란 비즈니스 포럼,2017-11-15,2017-11-15,신남방 중동팀,신청,확인,PT0200040351,1083037,2148137726
3,2017,PT,설명회,이란 비즈니스 포럼,2017-11-15,2017-11-15,신남방 중동팀,신청,확인,PT0200040419,EC139233,1198636524
4,2017,PT,설명회,전략시장 진출전략 설명회(대구),2017-12-18,2017-12-18,신남방 중동팀,신청,참가,PT0200040817,EC124347,6212099685


In [7]:
biz_df['수행부서'].value_counts()

베이징무역관           26109
상하이무역관           14374
도쿄무역관            11605
호치민무역관           10554
하노이무역관           10367
                 ...  
실리콘밸리IT해외지원센터        2
후속사업팀                2
정부간거래관리실             1
다마스커스무역관             1
공공조달팀                1
Name: 수행부서, Length: 212, dtype: int64

In [8]:
biz_df['기준년도'].value_counts()  # 2020, 2021, 2022 만 대상으로..

2021    147523
2020    140094
2019    122634
2018    115150
2022    101799
2017     27700
2023      9394
2016       357
2015         2
Name: 기준년도, dtype: int64

In [12]:
biz_df[biz_df['기준년도'].isin(["2020", "2021", "2022", "2023"])].to_csv("../Organization/biz_data.csv", index = False)

In [40]:
# 수행부서 이름을 넣으면 해당부서 사업과 참가기업이 나오는 함수

def show_org_biz(team, year = "2022"):
    df = biz_df[biz_df['수행부서'].notnull()]
    df = df[df['수행부서'].str.contains(team)]
    df = df[df['기준년도'].isin([year])]
    
    df = df[['기준년도','사업구분코드', '사업구분', '사업명', '수행부서', '사업시작일', '사업종료일', '참가기업사업자번호']].reset_index()
    df = df.drop('index', axis = 1)
    
    return df

In [41]:
show_org_biz('베이징')

,기준년도,사업구분코드,사업구분,사업명,수행부서,사업시작일,사업종료일,참가기업사업자번호
0,2022,RA,전시회,항목별시장조사,베이징무역관,2022-01-04,NaT,5518800506
1,2022,OB,기타사업,2022 중국 동계올림픽 연계 강원도 우수상품 전시상담회,베이징무역관,2022-01-01,2022-12-31,2248135176
2,2022,OB,기타사업,2022 중국 동계올림픽 연계 강원도 우수상품 전시상담회,베이징무역관,2022-01-01,2022-12-31,5484100728
3,2022,OB,기타사업,2022 중국 동계올림픽 연계 강원도 우수상품 전시상담회,베이징무역관,2022-01-01,2022-12-31,2211348498
4,2022,OB,기타사업,2022 중국 동계올림픽 연계 강원도 우수상품 전시상담회,베이징무역관,2022-01-01,2022-12-31,7218800352
...,...,...,...,...,...,...,...,...
2756,2022,OB,기타사업,해외IT지원센터운영(ICT성장산업실),베이징무역관,2022-11-16,2022-11-18,2073001965
2757,2022,OB,기타사업,내몽고 한국 우수상품 판촉주간,베이징무역관,2022-11-11,2022-11-13,5130398700
2758,2022,OB,기타사업,내몽고 한국 우수상품 판촉주간,베이징무역관,2022-11-11,2022-11-13,2474000395
2759,2022,OB,기타사업,BI(뉴트리패밀리주식회사)_PEK,베이징 BI,2022-09-01,2023-01-31,4518101225


In [75]:
def collect2(sql):
    conn = pyodbc.connect('DSN=BP_DB2;UID=bpuser;PWD=kotrabp')   # BPUSER2 에서 데이터 가져옴  // 만약 BPUSER1에서 데이터를 가져올경우 'DSN=BP_DB2'로 변경
    conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    conn.setencoding(encoding='utf-8')
    data= pd.read_sql_query(sql, conn)
    conn.close()
    return data

In [76]:
def make_expamt_data():
    sql = '''
SELECT c.BSNO_DECRYPT as BSNO, D.CONAME, B.COUNTRY_NM , A.EXPYR , A.EXIMAMTSUM 
FROM BPUSER.TB_CM_EXIMHIST_COUNTRY A, BPUSER.AP_DS_COUNTRY B, BPUSER.TB_CM_EXIMCORP_TOTAL C, BPUSER.TB_CM_CORP D
WHERE A.COUNTRYCD = b.COUNTRY_CD AND 
A.EXIMCOID = C.EXIMCOID AND 
c.BSNO_DECRYPT = D.BSNO AND
A.EXPYR >= '2020';
'''
    data = collect2(sql)
    return data
    

In [77]:
exp_data = make_expamt_data()

In [78]:
exp_data['EXPYR'].value_counts()

2020    216843
2021    216087
2022    209851
2023     70906
Name: EXPYR, dtype: int64

In [82]:
# 수행부서 이름과, 연도를 넣으면, 사업자번호를 추출해 주고, 해당 사업자번호별 2020년에서 2022년 전세계 수출금액 데이터를 추출해주는 함수

def show_org_corp_exp(team, year ='2022'):
    df = biz_df[biz_df['수행부서'].notnull()]
    df = df[df['수행부서'].str.contains(team)]
    df = df[df['기준년도'].isin([year])]
    
    bsno_df = df[['참가기업사업자번호']].drop_duplicates().reset_index()
    bsno_df = bsno_df.drop('index', axis = 1)
    bsno_df.columns = ['BSNO']
    bsno_df = bsno_df.merge(exp_data[exp_data['EXPYR'] == year], how ='left')
    bsno_df = bsno_df[['BSNO', 'CONAME', 'EXPYR', 'COUNTRY_NM', 'EXIMAMTSUM']]
    bsno_df = bsno_df[bsno_df['CONAME'].notnull()]
    
    
    return bsno_df
    


In [83]:
show_org_corp_exp('베이징')

,BSNO,CONAME,EXPYR,COUNTRY_NM,EXIMAMTSUM
0,5518800506,(주)케이비엘러먼트,2022,중국,3213.0
1,2248135176,횡성인삼영농조합법인,2022,미국,2574.0
2,2248135176,횡성인삼영농조합법인,2022,베트남,16854.0
3,5484100728,미제레레,2022,중국,19156.0
4,2211348498,티씨에스,2022,미국,1200.0
...,...,...,...,...,...
6049,6208148199,(주)엑소루브,2022,아랍에미리트,47868.0
6050,6208148199,(주)엑소루브,2022,러시아연방,274800.0
6051,6208148199,(주)엑소루브,2022,타지키스탄,101564.0
6052,6208148199,(주)엑소루브,2022,중국,111860.0


In [ ]:
# 어떻게 하면 사용자가 편하게 사용할 수 있을까??
# 부서명을 치고, 연도를 치면 해당사업수행 목록이 나오고.. 참가한 사업리스트를 확인할 수 있음, 다운로드 할 수 있음
